In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os

import numpy as np

from keras.models import load_model

import cv2

import pandas as pd

import time

In [0]:
model_path = '/content/drive/My Drive/TGS-Salt-Segmentation/models/trained/unet_101_binary_crossentropy_adam_accuracy.h5'
test_img_path = '/content/drive/My Drive/TGS-Salt-Segmentation/data/test/x_test.npy'

model_input_size = 101 if '101' in model_path else '128'

In [0]:
for root, dirs, test_img_names in os.walk(test_img_path):
    test_img_ids = [img_name[:-4] for img_name in test_img_names]
    test_img_paths = [test_img_path + img_name for img_name in test_img_names]

In [0]:
x_test = np.load(test_img_path)

In [6]:
model = load_model(model_path)

W0812 09:06:35.445056 139979139045248 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 09:06:35.586144 139979139045248 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0812 09:06:35.691186 139979139045248 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 09:06:35.692797 139979139045248 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0812 09:06:35.693882 139979139

In [7]:
x_test.shape

(18000, 101, 101, 1)

In [13]:
rle_masks = []
need_resize = model_input_size != 101
start = time.time()
for i in range(x_test.shape[0]):
    img = x_test[i]
    if need_resize:
        img = cv2.resize(img, (model_input_size , model_input_size), interpolation=cv2.INTER_CUBIC)
    img = img.reshape((1,model_input_size, model_input_size, 1))
    prediction = model.predict(img)
    prediction = (prediction > 0.7).astype(float)
    if need_resize:
        prediction = cv2.resize(prediction, (101,101), interpolation=cv2.INTER_AREA)
    prediction = prediction >= 0.99
    rle_masks.append(rle_encode(prediction))
    if i%100 == 0:
      print(time.time() - start)
      start = time.time()

0.12012743949890137
9.841862678527832
10.019863367080688
9.953751802444458
9.901506185531616
9.849047422409058
9.88873553276062
9.806111335754395
9.82355785369873
9.753323078155518
9.80635380744934
9.797557830810547


KeyboardInterrupt: ignored

In [0]:
def rle_encode(im):
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
#     print(runs, runs[1::2])
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [0]:
rle_masks = []
for img_path in test_img_paths:
    img = cv2.imread(img_path, 0)
    need_resize = model_input_size != 101
    if need_resize:
        img = cv2.resize(img, (model_input_size , model_input_size), interpolation=cv2.INTER_CUBIC)
    img = img.reshape((1,model_input_size, model_input_size, 1))
    prediction = model.predict(img)
    prediction = (prediction > 0.7).astype(float)
    if need_resize:
        prediction = cv2.resize(prediction, (101,101), interpolation=cv2.INTER_AREA)
    prediction = prediction >= 0.99
    rle_masks.append(rle_encode(prediction))
    

In [0]:
submission = {'id':test_img_ids, 'rle_mask':rle_masks}

submission_df = pd.DataFrame.from_dict(submission)
# save to csv